In [1]:
import pandas as pd
from tbdynamics.tools.plotting import plot_model_vs_actual
import nevergrad as ng
from tbdynamics.settings import CM_PATH
# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.camau.calibration.utils import get_bcm, load_targets
from multiprocessing import cpu_count

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"

In [3]:
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':False
}
bcm = get_bcm(params, covid_effects, homo_mixing=False)

In [4]:
# Set the number of workers for parallel optimization
orunner = optimize_model(bcm, opt_class=ng.optimizers.TwoPointsDE, num_workers=cpu_count())
for i in range(8):
    rec = orunner.minimize(500)
mle_params = rec.value[1]
mle_params

{'contact_rate': 0.02038856099685169,
 'rr_infection_latent': 0.1916528342209353,
 'rr_infection_recovered': 0.11096834328282291,
 'progression_multiplier': 1.226890849240191,
 'smear_positive_death_rate': 0.39027958571367316,
 'smear_negative_death_rate': 0.030164226790567385,
 'smear_positive_self_recovery': 0.23478457627790544,
 'smear_negative_self_recovery': 0.15203441964967288,
 'screening_scaleup_shape': 0.172075317897326,
 'screening_inflection_time': 1987.4679490565436,
 'time_to_screening_end_asymp': 3.7364884629014856,
 'acf_sensitivity': 0.892229345665419,
 'detection_reduction': 0.4777947982148881,
 'notif_dispersion': 65.24171544819497,
 'latent_dispersion': 9.750647367560333,
 'acf_detectionXact3_trial_dispersion': 42.69270423740469,
 'acf_detectionXact3_control_dispersion': 27.60997099113439}

In [5]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets(CM_PATH / 'targets.yml')

In [6]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [7]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [8]:
derived_df_0['detection_rate'].plot()

In [9]:
derived_df_0['mortality'].plot()

In [10]:
# derived_df_0['act3_trial_adults_pop'].plot()
# derived_df_0['act3_control_adults_pop'].plot()
# derived_df_0[[f'act3_{act3_stratum}_adults_pop' for act3_stratum in ["trial","control", "other"]]].plot()

In [11]:
derived_df_0[[f'total_populationXact3_{act3_stratum}' for act3_stratum in ["trial","control"]]].plot()

In [12]:
derived_df_0['acf_detectionXact3_trialXorgan_pulmonary'].plot()

In [13]:
derived_df_0['acf_detectionXact3_controlXorgan_pulmonary'].plot()